Используем модель Mistral 7B(Ollama)

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

class E8CompanyParser:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def parse_equipment_page(self, url: str) -> dict:
        """Парсинг страницы оборудования"""
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Основные данные
            data = {
                'name': self._extract_equipment_name(response.text),
                'price': self._extract_price(soup),
                'specs': self._extract_specs(soup),
                'description': self._get_text(soup, 'div.product__desc'),
                'images': self._extract_images(soup),
                'link': url
            }
            
            return data

        except Exception as e:
            print(f"Ошибка парсинга страницы {url}: {str(e)}")
            return {}

    def _extract_equipment_name(self, html: str) -> str:
        """Извлечение названия оборудования"""
        try:
            # Метод 1: Поиск в любом теге с id="pagetitle"
            pagetitle_match = re.search(r'<[^>]*id=["\']pagetitle["\'][^>]*>(.*?)</', html, re.DOTALL)
            if pagetitle_match:
                name = self._clean_text(pagetitle_match.group(1))
                if name and name != "== $0":
                    return name
            
            # Метод 2: Поиск в title страницы
            title_match = re.search(r'<title>(.*?)</title>', html, re.DOTALL)
            if title_match:
                title = self._clean_text(title_match.group(1))
                # Извлекаем наиболее вероятное название из title
                name_match = re.search(r'^(.*?)(?:\s*[|-]|\s*купить)', title)
                if name_match:
                    return name_match.group(1).strip()
                return title
            
            # Метод 3: Поиск в заголовке h1
            h1_match = re.search(r'<h1[^>]*>(.*?)</h1>', html, re.DOTALL)
            if h1_match:
                return self._clean_text(h1_match.group(1))
            
            return "Название не найдено"
            
        except Exception as e:
            return "Ошибка при извлечении названия"

    def _extract_specs(self, soup: BeautifulSoup) -> dict:
        """Извлечение характеристик"""
        specs = {}
        try:
            rows = soup.find_all('tr')
            for row in rows:
                name = row.find('td', class_='char_name') or row.find('th')
                value = row.find('td', class_='char_value') or row.find('td')
                if name and value:
                    key = self._clean_text(name.get_text())
                    val = self._clean_text(value.get_text())
                    if key and val:
                        specs[key] = val
        except Exception:
            pass
        return specs

    def _extract_price(self, soup: BeautifulSoup) -> str:
        """Извлечение цены"""
        try:
            price_block = soup.find('div', class_='product__price') or \
                         soup.find('span', class_='price') or \
                         soup.find('div', class_='price')
            if price_block:
                price = price_block.find('span') or price_block
                return self._clean_text(price.get_text())
            return "Цена не указана"
        except Exception:
            return "Ошибка при получении цены"

    def _extract_images(self, soup: BeautifulSoup) -> list:
        """Извлечение изображений"""
        images = []
        try:
            for img in soup.find_all('img'):
                src = img.get('src') or img.get('data-src')
                if src:
                    img_url = urljoin('https://e8company.ru', src)
                    if img_url not in images:
                        images.append(img_url)
        except Exception:
            pass
        return images

    @staticmethod
    def _get_text(soup: BeautifulSoup, selector: str) -> str:
        """Безопасное получение текста по селектору"""
        try:
            element = soup.select_one(selector)
            return E8CompanyParser._clean_text(element.get_text()) if element else ""
        except Exception:
            return ""

    @staticmethod
    def _clean_text(text: str) -> str:
        """Очистка текста"""
        try:
            if not text:
                return ""
            text = re.sub(r'==\s*\$\d+', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            return text
        except Exception:
            return ""

if __name__ == "__main__":
    parser = E8CompanyParser()
    test_url = "https://e8company.ru/catalog/teploobmenniki/e8_teploobmenniki/e8_s_32/"
    
    print("🔄 Парсим страницу оборудования...")
    equipment_data = parser.parse_equipment_page(test_url)
    
    print("\n✅ Результат парсинга:")
    print(f"Название: {equipment_data.get('name', 'Не удалось получить название')}")
    print(f"Цена: {equipment_data.get('price', 'Не удалось получить цену')}")
    print("Характеристики:")
    for key, value in equipment_data.get('specs', {}).items():
        print(f"  {key}: {value}")
    print(f"Описание: {equipment_data.get('description', 'Нет описания')[:200]}...")
    print(f"Изображения ({len(equipment_data.get('images', []))}): {equipment_data.get('images', [])[:2]}...")

🔄 Парсим страницу оборудования...

✅ Результат парсинга:
Название: Теплообменник Е8-S-32 (Ду 32)
Цена: 41 501 руб.
Характеристики:
  Материал: Углеродистая сталь (стандарт)
  Расчетная максимальная температура, °С: 150
  Ду теплообенника: 32
  Объем бака, л: 4,6
  Схема потоков: Одноходовой
  Среда по греющей стороне: Вода
  Среда по нагреваемой стороне: Вода
  Расход по греющей стороне: 11,2 тч
  Расход по нагреваемой стороне: 4,67 тч
  Температура на входе по греющей стороне, °С: 95
  Температура на входе по нагреваемой стороне, °С: 5
  Температура на выходе по греющей стороне, °С: 70
  Температура на выходе по нагреваемой стороне, °С: 65
  Потери давления по греющей стороне: 29,73 кПа
  Потери давления по нагреваемой стороне: 5,76 кПа
  Тепловая нагрузка: 280000 ккалч
  Запас площади поверхности, %: 16,1
  Коэф. теплопередачи: 4017
  Эффективная площадь, м2: 2
  Число пластин: 50
  Соединения: Резьбовое
  Покрытие портов: -
  Ответные фланцы: -
  Расчетное давление: 10
  Пробное дав

В данном коде используется модель GPT-2, русскоязычная версия от Сбербанка:

In [18]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from bs4 import BeautifulSoup
import requests
from typing import List, Dict, Optional
import re
import pandas as pd
from urllib.parse import urljoin

class WebEquipmentScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def scrape_equipment_data(self, base_url: str, search_query: str = None) -> List[Dict]:
        """
        Сбор данных об оборудовании с веб-сайта
        
        :param base_url: URL сайта для парсинга
        :param search_query: опциональный поисковый запрос
        :return: список словарей с данными оборудования
        """
        try:
            if search_query:
                search_url = f"{base_url}/search?q={search_query}"
                response = requests.get(search_url, headers=self.headers)
            else:
                response = requests.get(base_url, headers=self.headers)
            
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Здесь нужно адаптировать под конкретную структуру сайта
            equipment_items = soup.find_all('div', class_='equipment-item')  # Пример - измените под целевой сайт
            
            results = []
            for item in equipment_items:
                name = item.find('h3').text.strip() if item.find('h3') else "Неизвестное оборудование"
                description = item.find('div', class_='description').text.strip() if item.find('div', class_='description') else ""
                specs = item.find('ul', class_='specs').text.strip() if item.find('ul', class_='specs') else ""
                
                # Получаем полную ссылку на страницу оборудования
                relative_link = item.find('a')['href'] if item.find('a') else None
                full_link = urljoin(base_url, relative_link) if relative_link else ""
                
                # Дополнительный парсинг страницы оборудования
                if full_link:
                    detailed_data = self.scrape_equipment_details(full_link)
                else:
                    detailed_data = {}
                
                results.append({
                    'name': name,
                    'description': description,
                    'specifications': specs,
                    'link': full_link,
                    **detailed_data
                })
            
            return results
        
        except Exception as e:
            print(f"Ошибка при парсинге сайта: {e}")
            return []

    def scrape_equipment_details(self, url: str) -> Dict:
        """
        Парсинг детальной страницы оборудования
        
        :param url: URL страницы оборудования
        :return: словарь с дополнительными данными
        """
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Адаптируйте эти селекторы под структуру целевого сайта
            details = {
                'full_description': soup.find('div', class_='full-description').text.strip() if soup.find('div', class_='full-description') else "",
                'technical_specs': self.extract_technical_specs(soup),
                'images': [img['src'] for img in soup.select('.equipment-gallery img')] if soup.select('.equipment-gallery img') else [],
                'price': soup.find('span', class_='price').text.strip() if soup.find('span', class_='price') else "Не указана"
            }
            
            return details
        except Exception as e:
            print(f"Ошибка при парсинге детальной страницы: {e}")
            return {}

    @staticmethod
    def extract_technical_specs(soup: BeautifulSoup) -> Dict:
        """Извлечение технических характеристик в структурированном виде"""
        specs = {}
        spec_rows = soup.select('table.specs tr')  # Пример селектора - измените под целевой сайт
        
        for row in spec_rows:
            cells = row.find_all('td')
            if len(cells) == 2:
                key = cells[0].text.strip().rstrip(':')
                value = cells[1].text.strip()
                specs[key] = value
                
        return specs

class EquipmentDescriptionGenerator:
    def __init__(self, model_name: str = "sberbank-ai/rugpt3small_based_on_gpt2", use_web: bool = True):
        """
        Инициализация модели с возможностью веб-скрапинга
        
        :param model_name: название предобученной модели
        :param use_web: использовать ли данные из интернета для дополнения
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name).to(self.device)
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.model.resize_token_embeddings(len(self.tokenizer))
        
        self.web_scraper = WebEquipmentScraper() if use_web else None
        self.equipment_db = pd.DataFrame(columns=['name', 'description', 'specs', 'source'])
    
    def enrich_from_web(self, equipment_name: str, max_results: int = 3) -> None:
        """
        Поиск информации об оборудовании в интернете и добавление в базу знаний
        
        :param equipment_name: название оборудования для поиска
        :param max_results: максимальное количество результатов для сохранения
        """
        if not self.web_scraper:
            return
        
        # Примеры сайтов для парсинга (можно добавить больше)
        sites_to_scrape = [
            "https://www.medwow.com",
            "https://www.biomedical.com"
        ]
        
        for site in sites_to_scrape:
            results = self.web_scraper.scrape_equipment_data(site, equipment_name)
            for result in results[:max_results]:
                if result['name'].lower() in equipment_name.lower() or equipment_name.lower() in result['name'].lower():
                    description = result.get('full_description', result.get('description', ''))
                    specs = result.get('technical_specs', result.get('specifications', {}))
                    
                    new_entry = {
                        'name': result['name'],
                        'description': description,
                        'specs': str(specs),
                        'source': result['link']
                    }
                    
                    # Добавляем только если такого оборудования еще нет в базе
                    if not self.equipment_db[self.equipment_db['name'].str.contains(result['name'], case=False)].any().any():
                        self.equipment_db = pd.concat([self.equipment_db, pd.DataFrame([new_entry])], ignore_index=True)
    
    def generate_description(self, equipment_name: str, use_web_data: bool = True, 
                           max_length: int = 150, temperature: float = 0.7) -> str:
        """
        Генерация описания оборудования с возможностью использования веб-данных
        
        :param equipment_name: название оборудования
        :param use_web_data: использовать ли данные из интернета
        :param max_length: максимальная длина описания
        :param temperature: параметр "творчества"
        :return: сгенерированное описание
        """
        if use_web_data and self.web_scraper:
            self.enrich_from_web(equipment_name)
        
        # Проверяем, есть ли информация в нашей базе
        db_info = self.equipment_db[self.equipment_db['name'].str.contains(equipment_name, case=False)]
        
        if not db_info.empty:
            # Используем найденные данные как контекст для модели
            context = f"Технические характеристики оборудования {equipment_name}:\n"
            context += db_info.iloc[0]['specs'] + "\n\nОписание: " + db_info.iloc[0]['description']
            prompt = f"На основе следующей информации создай подробное описание оборудования:\n{context}\n\nПолное описание:"
        else:
            prompt = f"Оборудование: {equipment_name}\nПодробное описание:"
        
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.model.generate(
                input_ids,
                max_length=max_length,
                temperature=temperature,
                top_k=50,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True,
                no_repeat_ngram_size=2
            )
        
        description = self.tokenizer.decode(output[0], skip_special_tokens=True)
        description = description.replace(prompt, "").strip()
        
        return description

# Пример использования
if __name__ == "__main__":
    # Инициализация модели с веб-скрапингом
    generator = EquipmentDescriptionGenerator(use_web=True)
    
    equipment_names = [
        "Микроскоп Olympus CX23",
        "Центрифуга Eppendorf 5424",
        "Анализатор гематологический Mindray BC-2800"
    ]
    
    for name in equipment_names:
        print(f"\nГенерация описания для: {name}")
        
        # Генерация с использованием веб-данных
        web_description = generator.generate_description(name, use_web_data=True)
        print(f"Описание с веб-данными:\n{web_description}")
        
        # Генерация без использования веб-данных
        local_description = generator.generate_description(name, use_web_data=False)
        print(f"\nОписание без веб-данных:\n{local_description}")

C:\Users\0001\anaconda3\envs\deeppavlov_venv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Генерация описания для: Микроскоп Olympus CX23
Ошибка при парсинге сайта: HTTPSConnectionPool(host='www.biomedical.com', port=443): Max retries exceeded with url: /search?q=%D0%9C%D0%B8%D0%BA%D1%80%D0%BE%D1%81%D0%BA%D0%BE%D0%BF%20Olympus%20CX23 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B4F4801420>, 'Connection to www.biomedical.com timed out. (connect timeout=None)'))
Описание с веб-данными:
Встроенное устройство
Процессор: Cortex A3
Двигатель: 2,5 л.с.
Мощность: 4,3 л
Макс. частота: 8 Гц
Диапазон рабочих температур: от -20 до +45 °С
Время автономной работы: 30 минут
Объем: 720 мл
Вес: 3,1 кг
Габариты: 960x720x970 мм
Примечание:
Для сравнения: MacBook Pro 940x655 (MacBoy) 970x450 (Ubunt

Описание без веб-данных:
Olympa C-Master и Olympio CM-3, или, как их еще называют, «флагманские» объективы Olympiac Olympo.

«Флагманы» – объектива, в которых используются сверхпрочные матрицы. Их основная задача – обеспечивать высокую степень сжатия изображе

Код с использованием TF-IDF для генерации описаний оборудования на основе характеристик:

In [20]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class E8CompanyParser:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def parse_equipment_page(self, url: str) -> dict:
        """Парсинг страницы оборудования"""
        try:
            response = requests.get(url, headers=self.headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            data = {
                'name': self._get_text(soup, 'h1.product__title'),
                'price': self._extract_price(soup),
                'specs': self._extract_specs(soup),
                'description': self._get_text(soup, 'div.product__desc'),
                'images': self._extract_images(soup),
                'link': url
            }
            return data
        except Exception as e:
            print(f"Ошибка парсинга: {e}")
            return {}

    def _get_text(self, soup: BeautifulSoup, selector: str) -> str:
        """Безопасное извлечение текста по CSS-селектору"""
        element = soup.select_one(selector)
        return self._clean_text(element.text) if element else ""

    def _clean_text(self, text: str) -> str:
        """Очистка текста от лишних пробелов и переносов"""
        return ' '.join(text.strip().split()) if text else ""

    def _extract_price(self, soup: BeautifulSoup) -> str:
        """Извлечение цены"""
        price_block = soup.find('div', class_='product__price')
        if price_block:
            price = price_block.find('span')
            return self._clean_text(price.text) if price else "Цена по запросу"
        return "Цена не указана"

    def _extract_specs(self, soup: BeautifulSoup) -> dict:
        """Извлечение характеристик"""
        specs = {}
        names = soup.find_all('td', class_='char_name')
        values = soup.find_all('td', class_='char_value')
        
        if len(names) == len(values):
            for name, value in zip(names, values):
                key = self._clean_text(name.find('span', itemprop='name').text) if name.find('span', itemprop='name') else ""
                val = self._clean_text(value.find('span', itemprop='value').text) if value.find('span', itemprop='value') else ""
                if key and val:
                    specs[key] = val
        return specs

    def _extract_images(self, soup: BeautifulSoup) -> list:
        """Извлечение изображений"""
        images = []
        gallery = soup.find('div', class_='product__gallery')
        if gallery:
            for img in gallery.find_all('img'):
                if img.get('src'):
                    img_url = urljoin('https://e8company.ru', img['src'])
                    if img_url not in images:
                        images.append(img_url)
        return images

class TFIDFDescriber:
    def __init__(self):
        self.knowledge_base = [
            {"name": "Теплообменник E8-S32", "specs": {"Материал": "Углеродистая сталь", "Давление": "10 бар"}, 
             "description": "Пластинчатый теплообменник для систем отопления. Рабочее давление до 10 бар."},
            {"name": "Насос ABC-200", "specs": {"Мощность": "2.2 кВт", "Производительность": "200 л/мин"}, 
             "description": "Центробежный насос для воды с производительностью 200 литров в минуту."}
        ]
        self.vectorizer = TfidfVectorizer()

    def generate_description(self, equipment_data: dict) -> str:
        """Генерация описания через TF-IDF"""
        query_text = self._prepare_query(equipment_data)
        corpus = [self._prepare_query(item) for item in self.knowledge_base]
        
        tfidf_matrix = self.vectorizer.fit_transform(corpus + [query_text])
        similarities = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])
        best_match_idx = np.argmax(similarities)
        
        if similarities[0][best_match_idx] > 0.3:
            return self.knowledge_base[best_match_idx]["description"]
        else:
            return self._generate_default_description(equipment_data)

    def _prepare_query(self, data: dict) -> str:
        """Подготовка текста для TF-IDF"""
        specs_text = " ".join([f"{k}_{v}" for k, v in data['specs'].items()])
        return f"{data['name']} {specs_text}"

    def _generate_default_description(self, data: dict) -> str:
        """Генерация описания по умолчанию"""
        specs_text = "\n".join([f"- {k}: {v}" for k, v in data['specs'].items()])
        return f"""Оборудование: {data['name']}
        
Основные характеристики:
{specs_text}

Описание: промышленное оборудование для специализированных применений."""

if __name__ == "__main__":
    # Парсинг данных
    parser = E8CompanyParser()
    test_url = "https://e8company.ru/catalog/teploobmenniki/e8_teploobmenniki/e8_s_32/"
    print("🔄 Парсим страницу оборудования...")
    equipment_data = parser.parse_equipment_page(test_url)
    
    if not equipment_data:
        print("❌ Ошибка: не удалось получить данные")
        exit()

    # Генерация описания
    print("🔍 Генерируем описание через TF-IDF...")
    describer = TFIDFDescriber()
    description = describer.generate_description(equipment_data)
    
    print("\n✅ Результат:")
    print(description)

🔄 Парсим страницу оборудования...
🔍 Генерируем описание через TF-IDF...

✅ Результат:
Оборудование: 
        
Основные характеристики:
- Материал: Углеродистая сталь (стандарт)
- Расчетная максимальная температура, °С: 150
- Ду теплообенника: 32
- Объем бака, л: 4,6
- Схема потоков: Одноходовой
- Среда по греющей стороне: Вода
- Среда по нагреваемой стороне: Вода
- Расход по греющей стороне: 11,2 тч
- Расход по нагреваемой стороне: 4,67 тч
- Температура на входе по греющей стороне, °С: 95
- Температура на входе по нагреваемой стороне, °С: 5
- Температура на выходе по греющей стороне, °С: 70
- Температура на выходе по нагреваемой стороне, °С: 65
- Потери давления по греющей стороне: 29,73 кПа
- Потери давления по нагреваемой стороне: 5,76 кПа
- Тепловая нагрузка: 280000 ккалч
- Запас площади поверхности, %: 16,1
- Коэф. теплопередачи: 4017
- Эффективная площадь, м2: 2
- Число пластин: 50
- Соединения: Резьбовое
- Покрытие портов: -
- Ответные фланцы: -
- Расчетное давление: 10
- Пробное

In [14]:
import pandas as pd
from IPython.display import display

# Создаем DataFrame с данными
data = {
    'Метрика': ['Качество генерации', 'Консистентность', 'Скорость работы', 'Требования к GPU', 
                'Обучение', 'Кастомизация', 'Точность', 'Плюсы', 'Минусы'],
    'GPT-2 (Sber)': ['Среднее (есть артефакты)', '6/10', '1-2 сек', '4GB+', 
                     'Предобученная', 'Средняя', '65-70%', 
                     'Быстрая, легкая', 'Устаревшая архитектура'],
    'TF-IDF': ['Низкое (шаблонные тексты)', '4/10', '<0.5 сек', 'Нет', 
               'Не требуется', 'Высокая', '50-60%', 
               'Простота', 'Примитивность'],
    'Mistral (Ollama)': ['Высокое', '9/10', '3-10 сек (зависит от GPU)', '16GB+', 
                         'Предобученная', 'Высокая', '80-85%', 
                         'Качество генерации', 'Требует ресурсов']
}

df = pd.DataFrame(data)

# Функция для цветового выделения
def highlight_cells(val):
    colors = {
        'Высокое': 'background-color: #4CAF50; color: white;',
        'Среднее (есть артефакты)': 'background-color: #FFC107;',
        'Низкое (шаблонные тексты)': 'background-color: #F44336; color: white;',
        '9/10': 'background-color: #4CAF50; color: white;',
        '6/10': 'background-color: #FFC107;',
        '4/10': 'background-color: #F44336; color: white;',
        '80-85%': 'background-color: #4CAF50; color: white;',
        '65-70%': 'background-color: #FFC107;',
        '50-60%': 'background-color: #F44336; color: white;'
    }
    return colors.get(val, '')

# Применяем стили
styled_df = df.style\
    .set_properties(**{'text-align': 'left', 'font-size': '12pt'})\
    .set_table_styles([
        {'selector': 'th', 'props': [('font-size', '12pt'), ('background-color', '#607D8B'), ('color', 'white')]}
    ])\
    .applymap(highlight_cells, subset=pd.IndexSlice[:, ['GPT-2 (Sber)', 'TF-IDF', 'Mistral (Ollama)']])\
    .hide(axis='index')\
    .format(precision=2)

# Отображаем таблицу
display(styled_df)

# Добавляем текстовый вывод с рекомендациями
print("\nРекомендации:")
print("1. Для максимального качества → Mistral 7B (требуется GPU 16GB+)")
print("2. Для баланса скорости/качества → GPT-2 (Sber)")
print("3. Для простых задач → TF-IDF + шаблоны")

Метрика,GPT-2 (Sber),TF-IDF,Mistral (Ollama)
Качество генерации,Среднее (есть артефакты),Низкое (шаблонные тексты),Высокое
Консистентность,6/10,4/10,9/10
Скорость работы,1-2 сек,<0.5 сек,3-10 сек (зависит от GPU)
Требования к GPU,4GB+,Нет,16GB+
Обучение,Предобученная,Не требуется,Предобученная
Кастомизация,Средняя,Высокая,Высокая
Точность,65-70%,50-60%,80-85%
Плюсы,"Быстрая, легкая",Простота,Качество генерации
Минусы,Устаревшая архитектура,Примитивность,Требует ресурсов



Рекомендации:
1. Для максимального качества → Mistral 7B (требуется GPU 16GB+)
2. Для баланса скорости/качества → GPT-2 (Sber)
3. Для простых задач → TF-IDF + шаблоны
